In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = "1" #please put your GPU

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# %cd 'drive/MyDrive/Image Perception (Personal)'

In [4]:
# !dir

In [5]:
import tensorflow as tf
# import tensorflow_addons as tfa
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, MaxPool2D, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, Adagrad, SGD
from tensorflow.keras.applications import InceptionV3
from keras.regularizers import l2
# from io_ import score
import matplotlib.pyplot as plt

In [6]:
tf.__version__

'2.4.1'

## Splitting data into train, test, val 

In [7]:
# data_dir = 'dataset'

In [8]:
# !pip install split-folders tqdm

In [9]:
# import splitfolders  # or import split_folders  #split into train, test, val

# # Split with a ratio.
# # To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
# splitfolders.ratio(data_dir, output="Splitteddata/", seed=1337, ratio=(.8, .1, .1), group_prefix=None) # default values


## Loading data & data preprocessing

In [10]:
imagedatagenerator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20, 
    zoom_range=0.15,
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=0.15,
    horizontal_flip=True, 
    fill_mode="nearest", rescale=1.0/255.0)

In [11]:
batch_size = 128
img_height = 225
img_width = 225

In [12]:
train_dir = 'Splitteddata/train'
val_dir = 'Splitteddata/val'
test_dir = 'Splitteddata/test'

In [13]:
#training and validation (to be splitted later on)
train_ds = imagedatagenerator.flow_from_directory(
                                                train_dir,
                                                target_size=(img_height, img_width),
                                                color_mode="rgb",
                                                classes=None,
                                                class_mode="sparse",
                                                batch_size= batch_size,
                                                shuffle=True,
                                                seed=123,
                                                save_to_dir='Augmented Images',
                                                save_prefix="",
                                                save_format="jpg",
                                                follow_links=False,
                                                interpolation="nearest",
                                            )

Found 24186 images belonging to 72 classes.


In [14]:

# for _ in range(5):
#     img, label = train_ds.next()
#     print(img.shape)
#     print(img)#  (1,256,256,3)
#     plt.imshow(img[0])
#     plt.show()

In [15]:
val_ds = imagedatagenerator.flow_from_directory(
                                                val_dir,
                                                target_size=(img_height, img_width),
                                                color_mode="rgb",
                                                classes=None,
                                                class_mode="sparse",
                                                batch_size= batch_size,
                                                shuffle=True,
                                                seed=123,
                                                save_to_dir='Augmented Images',
                                                save_prefix="",
                                                save_format="jpg",
                                                follow_links=False,
                                                interpolation="nearest",
                                            )

Found 2999 images belonging to 72 classes.


## Model and Training of model

In [16]:
num_classes = 72

# #MODEL 3
# Write your code here
inceptionv3 = InceptionV3(input_shape = (225,225,3), weights='imagenet', include_top=False)
model = Sequential()
model.add(inceptionv3)
model.add(Dropout(0.4))
model.add(Flatten())

model.add(Dense(
    units= num_classes, 
    kernel_initializer='glorot_uniform', 
    kernel_regularizer=l2(.0005),
    activation='softmax'))


# model.add(Dense(units=72, activation="softmax"))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 2048)        0         
_________________________________________________________________
flatten (Flatten)            (None, 51200)             0         
_________________________________________________________________
dense (Dense)                (None, 72)                3686472   
Total params: 25,489,256
Trainable params: 25,454,824
Non-trainable params: 34,432
_________________________________________________________________


In [17]:
# Define your optimizer here
# optim = SGD(lr=1e-3)

# model.compile(loss='sparse_categorical_crossentropy',
#               optimizer=optim,
#               metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k = 1), 'accuracy'])


model.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer=SGD(lr=0.01, momentum=0.9),
    metrics=['accuracy', tf.keras.metrics.SparseTopKCategoricalAccuracy(k = 5)])

In [18]:
# save the model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='siti(27).h5', 
                                                mode='min', 
                                                monitor='val_loss', 
                                                verbose=2, 
                                                save_best_only=True)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=2,
                          verbose=2,
                          restore_best_weights=True
                          )


reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=1,
                              verbose=1,
                              min_delta=0.0001)

callbacks_list = [checkpoint,
                  earlystop,
                  reduce_lr]

epochs = 3
batch_size = 128

history = model.fit(train_ds, verbose = 1, # split data in 80-20 sets
                    epochs = epochs,
                    validation_data = val_ds,
                    batch_size = batch_size, callbacks = callbacks_list)



Epoch 1/3
189/189 [==============================] - 2678s 14s/step - loss: 3.2361 - accuracy: 0.3194 - sparse_top_k_categorical_accuracy: 0.5620 - val_loss: 3.2565 - val_accuracy: 0.2454 - val_sparse_top_k_categorical_accuracy: 0.5905

Epoch 00001: val_loss improved from inf to 3.25647, saving model to siti(27).h5
Epoch 2/3
189/189 [==============================] - 2746s 14s/step - loss: 0.9859 - accuracy: 0.7314 - sparse_top_k_categorical_accuracy: 0.9522 - val_loss: 1.2541 - val_accuracy: 0.6822 - val_sparse_top_k_categorical_accuracy: 0.9256

Epoch 00002: val_loss improved from 3.25647 to 1.25409, saving model to siti(27).h5
Epoch 3/3
189/189 [==============================] - 2420s 13s/step - loss: 0.6412 - accuracy: 0.8336 - sparse_top_k_categorical_accuracy: 0.9835 - val_loss: 1.1664 - val_accuracy: 0.7039 - val_sparse_top_k_categorical_accuracy: 0.9386

Epoch 00003: val_loss improved from 1.25409 to 1.16638, saving model to siti(27).h5


## Saving Best Model 

In [19]:
# save your model and weight (only submit best model)
best_model = model
model_json = best_model.to_json()
with open("Siti(27).json", "w") as json_file:
    json_file.write(model_json)
    
best_model.save_weights('Siti(27).h5')

In [ ]:
history

In [ ]:
history.history